# LangChain with Ollama

Steps followed from this: https://python.langchain.com/v0.1/docs/get_started/quickstart

### Imports

In [2]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


### LLM

initialize the LLM to be used along with an empty chat history

In [7]:
llm = Ollama(model="llama3")
chat_history = []

### Documents for Retrieval w/ Embeddings

get a couple documents and create some context. creates the embeddings and stores them in a vector space

In [3]:
loader = WebBaseLoader([
    "https://en.wikipedia.org/wiki/Gravity",
    "https://en.wikipedia.org/wiki/Consciousness"
])

docs = loader.load()
embeddings = OllamaEmbeddings(
    model="llama3"
)
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

### Invoke the model

In [8]:
# lvl 4
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

chat_history = [HumanMessage(content="Are there philosophical and scienctific crossovers in our current understandings of gravity and consciousness?"), AIMessage(content="Yes!")]

# first, we need to generate a query that can be used to look up relevant documents
# this is the new, updated retriever
retriever = vector.as_retriever()
retriever_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, retriever_prompt)

# test it out if desired
# retriever_chain.invoke({
#     "chat_history": chat_history,
#     "input": "Tell about them"
# })

# second, we generate an answer to the user's original question
reply_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, reply_prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

# invoke the llm to answer the question
q = "Tell me about them"
a = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": q
})["answer"]
print(a)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# lvl 3 - use the context provided by the document retrieval to answer the question
# create a prompt that can look up relevant information in a document vectorstore
# prompt = ChatPromptTemplate.from_template("""
# You are a world class astrophysicist who speciallizes in gravity and dabbles in consciousness for fun. Answer the following question using the provided context to enhance your answer:

# <context>
# {context}
# </context>

# Question: {input}""")

# # now create the retrieval chain
# document_chain = create_stuff_documents_chain(llm, prompt)
# retriever = vector.as_retriever()
# retrieval_chain = create_retrieval_chain(retriever, document_chain)

# # and invoke it
# q = "How is consciousness related to gravity?"
# a = retrieval_chain.invoke(
#     {"input": q}
# )["answer"]
# print(a)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# lvl 2 - provide a system prompt and get the output
# output_parser = StrOutputParser()
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a world class astrophysicist who speciallizes in gravity and dabbles in consciousness for fun."),
#     ("user", "{input}")
# ])
# chain = prompt | llm | output_parser
# q = "How is consciousness related to gravity?"
# a = chain.invoke({
#     "input": q
# })

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# lvl 1 - no context, bare bones
# q = "what is gravity?"
# a = llm.invoke("what is gravity?")
# print(a)

# handle the chat history
chat_history.append(
    HumanMessage(content=q)
)
chat_history.append(
    AIMessage(content=a)
)

What a fascinating intersection! While gravity is a well-studied force in physics, consciousness remains a complex and multifaceted topic in philosophy, neuroscience, and psychology. Despite these differences, there are intriguing crossovers between our understanding of gravity and consciousness.

One significant connection lies in the concept of emergence. In both cases, we're dealing with phenomena that arise from the interactions and organization of individual components. Gravity emerges from the collective behavior of massive objects warping spacetime, while consciousness likely arises from the intricate workings of neurons, glial cells, and other brain elements interacting and processing information.

Another crossover involves the notion of complexity. Gravity's effects become apparent at certain scales (e.g., planetary or cosmic), whereas consciousness is typically associated with complex biological systems like brains. However, both phenomena exhibit self-organizing properties,